# Classificação: aprendizado semi-supervisionado

Uma indústria de laticínios precisa assegurar a qualidade do leite para produzir seus produtos.

Neste projeto, iremos analisar características de amostras de leite para identificar a qualidade e verificar se podem ser utilizados na produção dos produtos alimentícios.

O machine learning será utilizado para fazer a **classificação** do leite entre as qualidades **baixa, média e alta**.

## Carregando os dados

In [2]:
import pandas as pd

path = "dados/qualidade_leite.csv"
df_dados = pd.read_csv(path)
df_dados

,pH,Temperatura,Sabor,Odor,Gordura,Turbidez,Cor,Qualidade
0,6.6,35,1,0,1,0,254,alta
1,6.6,36,0,1,0,1,253,alta
2,8.5,70,1,1,1,1,246,NaN
3,9.5,34,1,1,0,1,255,NaN
4,6.6,37,0,0,0,0,255,média
...,...,...,...,...,...,...,...,...
1054,6.7,45,1,1,0,0,247,NaN
1055,6.7,38,1,0,1,0,255,NaN
1056,3.0,40,1,1,1,1,255,NaN
1057,6.8,43,1,0,1,0,250,NaN
